In [172]:
import pandas as pd

boi = pd.read_excel('planilha_dos_boi.xlsx', sheet_name='plms')
geral = pd.read_excel('planilha_dos_boi.xlsx', sheet_name='Brutos')
boi.dropna(axis=1, how='all', inplace=True)
boi.dropna(axis=0, how='all', inplace=True)

In [ ]:
boi

In [173]:
geral = geral[['SERIAL BOX', 'Data correta', 'BATERIA']]
geral.rename(columns={'SERIAL BOX':'PLM'}, inplace=True)

In [ ]:
geral.head()

In [ ]:
boi

In [174]:
boi.iloc[0, :] = boi.iloc[0, :].fillna('Bateria')
boi.set_axis(labels=boi.iloc[0, :], axis=1, inplace=True)
boi.drop(axis=0, labels=1, inplace=True)

C:\Users\guilherme.costa\AppData\Local\Temp\ipykernel_9512\1022803827.py:2: FutureWarning:

DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead



In [175]:
colunas_plm = [coluna for coluna in boi.columns if 'plm' in coluna.lower()]
df = pd.DataFrame(colunas_plm)

In [176]:
df.rename(columns={0:'PLM'}, inplace=True)

In [ ]:
df

In [177]:
concatenado = df.merge(geral, how='inner', on='PLM')
concatenado

,PLM,Data correta,BATERIA
0,27901699001PLM0208190056,16/05/2023 08:28:32,4166.00
1,27901699001PLM0208190056,16/05/2023 02:28:47,4164.00
2,27901699001PLM0208190056,16/05/2023 20:57:53,4168.00
3,27901699001PLM0208190056,15/05/2023 20:28:29,4174.00
4,27901699001PLM0208190056,15/05/2023 08:28:36,4166.00
...,...,...,...
1150,27901699001PLM0208190328,10/05/2023 20:13:20,3904.00
1151,27901699001PLM0208190328,10/05/2023 15:12:15,3898.00
1152,27901699001PLM0208190328,03/04/2023 21:54:21,3392.00
1153,27901699001PLM0208190328,03/04/2023 09:54:21,3.39


In [178]:
concatenado.to_excel('novo_concatenado.xlsx', index=False)

In [179]:
conc = pd.read_excel('novo_concatenado.xlsx')
conc.dropna(axis=0, how='any', inplace=True)

In [180]:
from datetime import datetime
conc['Data correta'] = conc['Data correta'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

In [168]:
# conc['Data'] = conc['Data correta'].astype(str)
# conc['Data'] = conc['Data Data'].apply(lambda x: x[5:10])
conc

,PLM,Data correta,BATERIA
0,27901699001PLM0208190056,2023-05-16 08:28:32,4166.00
1,27901699001PLM0208190056,2023-05-16 02:28:47,4164.00
2,27901699001PLM0208190056,2023-05-16 20:57:53,4168.00
3,27901699001PLM0208190056,2023-05-15 20:28:29,4174.00
4,27901699001PLM0208190056,2023-05-15 08:28:36,4166.00
...,...,...,...
1150,27901699001PLM0208190328,2023-05-10 20:13:20,3904.00
1151,27901699001PLM0208190328,2023-05-10 15:12:15,3898.00
1152,27901699001PLM0208190328,2023-04-03 21:54:21,3392.00
1153,27901699001PLM0208190328,2023-04-03 09:54:21,3.39


In [181]:
import plotly.express as px
import plotly.graph_objects as go
agrupado = conc.groupby(by='PLM')
grupo = agrupado.get_group('27901699001PLM0208190056')
grupo1 = agrupado.get_group('27901699001PLM0208190328')
grupos = grupo[['PLM', 'BATERIA','Data correta']]
grupos1 = grupo1[['PLM', 'BATERIA','Data correta']]


In [184]:
def multiplica(bateria):
    if bateria < 100:
        bateria *= 1000
    return bateria

fig = go.Figure()
for name, grupo in agrupado:
    grupo.sort_values(by='Data correta', inplace=True)
    grupo['BATERIA'] = grupo['BATERIA'].apply(lambda x: multiplica(x))
    grupo = grupo[grupo['Data correta'].dt.month > 4]
    fig.add_trace(go.Scatter(x=grupo['Data correta'], y=grupo['BATERIA'], mode="markers+lines", line_shape='spline', name=name))
    # fig.add_trace(go.Scatter(x=grupo['Data correta'], y=grupo['BATERIA'], mode="markers+lines", line_shape='spline', legendgroup=name))
fig.update_layout(width=1500, height=800)

In [ ]:
px.line(data_frame=conc, x='Data correta', y='BATERIA')
